In [1]:
import numpy as np

In [3]:
# split a univariate sequence into samples
def split_sequences(sequences,n_steps_in,n_steps_out):
    X = list()
    Y = list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in #(0+3,1+3,2+3,4+3,5+3)
        out_end_ix = end_ix + n_steps_out #(3+2,4+2,5+2,6+2,7+2)
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix]
        seq_y = sequences[end_ix:out_end_ix]
        X.append(seq_x)
        Y.append(seq_y)
    return np.array(X),np.array(Y)

In [4]:
# define input sequence
sequence = [10,20,30,40,50,60,70,80,90]

In [5]:
n_steps = 3
n_steps_out = 2

In [6]:
X,Y = split_sequences(sequence,n_steps,n_steps_out)

In [7]:
X

array([[10, 20, 30],
       [20, 30, 40],
       [30, 40, 50],
       [40, 50, 60],
       [50, 60, 70]])

In [8]:
X.shape

(5, 3)

In [9]:
Y

array([[40, 50],
       [50, 60],
       [60, 70],
       [70, 80],
       [80, 90]])

In [10]:
Y.shape

(5, 2)

In the case of the Encoder-Decoder model, the output, or y part, of the training dataset must also have this shape.


This is because the model will predict a given number of time steps with a given number of features for each input sample.

In [11]:
n_features = 1

In [12]:
X = X.reshape(X.shape[0],X.shape[1],n_features)
Y = Y.reshape(Y.shape[0],Y.shape[1],n_features)

In [13]:
X

array([[[10],
        [20],
        [30]],

       [[20],
        [30],
        [40]],

       [[30],
        [40],
        [50]],

       [[40],
        [50],
        [60]],

       [[50],
        [60],
        [70]]])

In [14]:
X.shape

(5, 3, 1)

In [15]:
Y.shape

(5, 2, 1)

In [16]:
Y

array([[[40],
        [50]],

       [[50],
        [60]],

       [[60],
        [70]],

       [[70],
        [80]],

       [[80],
        [90]]])

# Model Building

In [56]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

# Encoder-Decoder Model
A model specifically developed for forecasting variable length output sequences is called the Encoder-Decoder LSTM.

The model was designed for prediction problems where there are both input and output sequences, so-called sequence-to-sequence, or seq2seq problems, such as translating text from one language to another.

As its name suggests, the model is comprised of two sub-models: the encoder and the decoder.

The encoder is a model responsible for reading and interpreting the input sequence. The output of the encoder is a fixed length vector that represents the model’s interpretation of the sequence. 

The encoder is traditionally a Vanilla LSTM model, although other encoder models can be used such as Stacked, Bidirectional, and CNN models.

In [63]:
# define model
model = Sequential()
model.add(LSTM(100,activation = 'relu',input_shape = (n_steps,n_features)))

The decoder uses the output of the encoder as an input.

First, the fixed-length output of the encoder is repeated, once for each required time step in the output sequence.

In [64]:
model.add(RepeatVector(n_steps_out))

This sequence is then provided to an LSTM decoder model.

The model must output a value for each value in the output time step, which can be interpreted by a single output model

In [65]:
model.add(LSTM(100,activation = 'relu',return_sequences = True))


We can use the same output layer or layers to make each one-step prediction in the output sequence.

This can be achieved by wrapping the output part of the model in a TimeDistributed wrapper.

In [66]:
model.add(TimeDistributed(Dense(1)))

In [67]:
model.compile(optimizer = 'adam',loss = 'mse')

In [68]:
model.fit(X,Y,epochs=100)

Epoch 1/100
5/5 [==============================] - 3s 694ms/step - loss: 4427.1938
Epoch 2/100
5/5 [==============================] - 0s 2ms/step - loss: 4381.4443
Epoch 3/100
5/5 [==============================] - 0s 2ms/step - loss: 4335.0410
Epoch 4/100
5/5 [==============================] - 0s 2ms/step - loss: 4287.1729
Epoch 5/100
5/5 [==============================] - 0s 2ms/step - loss: 4235.8164
Epoch 6/100
5/5 [==============================] - 0s 2ms/step - loss: 4179.6689
Epoch 7/100
5/5 [==============================] - 0s 2ms/step - loss: 4115.9775
Epoch 8/100
5/5 [==============================] - 0s 1ms/step - loss: 4041.6770
Epoch 9/100
5/5 [==============================] - 0s 2ms/step - loss: 3952.9087
Epoch 10/100
5/5 [==============================] - 0s 1ms/step - loss: 3845.6694
Epoch 11/100
5/5 [==============================] - 0s 2ms/step - loss: 3716.0671
Epoch 12/100
5/5 [==============================] - 0s 1ms/step - loss: 3558.8804
Epoch 13/100
5/5 [=====

# Prediction

In [69]:
x_input = np.array([70,80,90])

In [70]:
x_input = x_input.reshape((1,n_steps,n_features))

In [71]:
model.predict(x_input)

array([[[102.07573 ],
        [113.712585]]], dtype=float32)